<a href="https://colab.research.google.com/github/SumaKori93/MasterThesisReport/blob/master/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 287.80 KiB | 8.22 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [ ]:
!pip install mxnet

     |████████████████████████████████| 25.4MB 118kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [ ]:
!pip install gluonnlp

     |████████████████████████████████| 245kB 2.8MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.8.2-cp36-none-any.whl size=293515 sha256=5eb35103e8c231c90ac6b0275fba2e37d000e54952eb3fdcf0c15dfa1dd23a14
  Stored in directory: /root/.cache/pip/wheels/a4/aa/61/0aebc5c078c4b1ccf325cd7579932b99403008da6e7ce6b68f
Successfully built gluonnlp


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from bert import data, model

In [ ]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.cpu(0)

In [ ]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='biobert_v1.0_pubmed_pmc_cased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
#print(bert_base)

Vocab file is not found. Downloading.


In [ ]:
#Attach classification layer. BERTclassifier uses BERT base model to encode sentence representation
bert_classifier = model.classification.BERTClassifier(bert_base, num_classes=3, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

In [ ]:
import io
tsv_file = io.open('/content/test_howdo.tsv', encoding='utf-8')
for i in range(10):
    print(tsv_file.readline())

	RELEVANCE	Paragraph	Question

14049	0	"&nbsp;&nbsp; Genetic factors Top Abstract Introduction Epidemiology Clinico-histopathology Genetic factors Infectious agents Experimentally induced gastric... Prospects for prevention of... Conclusions References &nbsp; Observational studies are useful to determine whether heredityplays a role in gastric cancer and molecular epidemiologicalstudies can be used to identify factors involved in cancer development,progression and metastasis. Studies aimed at determining therole of heredity in stomach cancer include family pedigree reports,twin studies, family aggregation studies and blood typing studies.As discussed by Nomura (33), it has been shown that gastriccancer can be present in families over two to three generationsand family members of a stomach cancer case have two to threetimes the risk than does the general population of developinggastric cancer. Studies involving family members are methodologicallylimited, however, because they do not dis

In [ ]:
# Skip the first line, which is the schema
# Skip the first line, which is the schema
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
#Fields to select from the file
field_indices = [2, 3, 1]
data_train_raw = nlp.data.TSVDataset(filename='/content/train_howdo.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples, field_indices=field_indices)
data_test_raw = nlp.data.TSVDataset(filename='/content/test_howdo.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples, field_indices=field_indices)

sample_id = 100
# Sentence A
print(data_train_raw[sample_id][0])
# Sentence B
print(data_train_raw[sample_id][1])
# 1 means equivalent, 0 means not equivalent
print(data_train_raw[sample_id][2])

"1. &nbsp; Arany, E, Zabel P, Freeman D, and Hill DJ. Elimination of radiolabelled recombinant human insulin-like growth factor binding protein-3 from the circulation, and its distribution amongst organs and tissues in adult male rats. Regul Pept 48: 133-143, 1993[ISI][Medline]. 2. &nbsp;"
How do interactions between insulin-like GFs and the insulin receptor affect skin biology?
0


In [ ]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=False)

In [ ]:
# The maximum length of an input sequence
max_len = 128
#  0 = not, 1 = possibly,  2=definitely
#all_labels = ['NOT','POSSIBLY', 'DEFINITELY']
all_labels = ['0', '1', '2']

In [ ]:
pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len, 
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_train = data_train_raw.transform(transform)

In [ ]:

print('valid length = \n%s'%data_train[sample_id][1])
print('segment ids = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[14][3])

valid length = 
112
segment ids = 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
label = 
[0]


In [ ]:
print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))

vocabulary used for tokenization = 
Vocab(size=28996, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 0
[CLS] token id = 101
[SEP] token id = 102


In [ ]:
# The hyperparameters
batch_size = 8
lr = 3e-5
decay = 1e-6
momentum=0.9
nesterov=True

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})
#trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'sgd',
#                           {'learning_rate': lr, 'momentum': momentum})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 32
num_epochs = 2

In [ ]:
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0

[Epoch 0 Batch 32/1690] loss=0.4696, lr=0.0000300, acc=0.877
[Epoch 0 Batch 64/1690] loss=0.4274, lr=0.0000300, acc=0.874
[Epoch 0 Batch 96/1690] loss=0.3423, lr=0.0000300, acc=0.881
[Epoch 0 Batch 128/1690] loss=0.3615, lr=0.0000300, acc=0.884
[Epoch 0 Batch 160/1690] loss=0.4852, lr=0.0000300, acc=0.877
[Epoch 0 Batch 192/1690] loss=0.4128, lr=0.0000300, acc=0.874
[Epoch 0 Batch 224/1690] loss=0.3602, lr=0.0000300, acc=0.878
[Epoch 0 Batch 256/1690] loss=0.2737, lr=0.0000300, acc=0.882
[Epoch 0 Batch 288/1690] loss=0.3339, lr=0.0000300, acc=0.884
[Epoch 0 Batch 320/1690] loss=0.3729, lr=0.0000300, acc=0.882
[Epoch 0 Batch 352/1690] loss=0.3340, lr=0.0000300, acc=0.884
[Epoch 0 Batch 384/1690] loss=0.3007, lr=0.0000300, acc=0.885
[Epoch 0 Batch 416/1690] loss=0.3023, lr=0.0000300, acc=0.887
[Epoch 0 Batch 448/1690] loss=0.2833, lr=0.0000300, acc=0.887
[Epoch 0 Batch 480/1690] loss=0.4997, lr=0.0000300, acc=0.886
[Epoch 0 Batch 512/1690] loss=0.2298, lr=0.0000300, acc=0.888
[Epoch 0 Ba

In [ ]:
#Run test_set
data_test = data_test_raw.transform(transform)
test_dataloader = mx.gluon.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, last_batch='rollover')

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, zero_one_loss
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

def print_results(y_true, y_pred):
    print("Error Rate: {:2.2f}".format(100*zero_one_loss(y_true, y_pred)))
    print("Accuracy: {:2.2f}".format(100*accuracy_score(y_true, y_pred)))
    print("F1-Score: {:2.2f}".format(100*f1_score(y_true, y_pred, average="weighted")))
    print("Precision: {:2.2f}".format(100*precision_score(y_true, y_pred, average="weighted")))
    print("Recall: {:2.2f}".format(100*recall_score(y_true, y_pred, average="weighted")))
    #fig, ax = plt.subplots(figsize=(18, 10))
    #x = sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, ax=ax)
    #x.invert_yaxis()
    #plt.xlabel("Predicted")
    #plt.ylabel("True")
    #return fig

In [ ]:
from mxnet import gluon, nd, autograd
y_true = []
y_pred = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.as_in_context(ctx)
    valid_length = valid_length.as_in_context(ctx)
    segment_ids = segment_ids.as_in_context(ctx)
    label = label.as_in_context(ctx)
    out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
    pred = nd.argmax(out, axis=1).asnumpy()
    y_true.extend(list(np.reshape(label.asnumpy(), (-1))))
    y_pred.extend(pred)
assert len(y_true)==len(y_pred)

In [ ]:
fig = print_results(np.reshape(y_true, (-1)), y_pred)
from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred))
print(confusion_matrix(y_true,y_pred))

Error Rate: 8.17
Accuracy: 91.83
F1-Score: 91.49
Precision: 91.20
Recall: 91.83
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      2991
           1       0.42      0.34      0.37       157
           2       0.65      0.60      0.63       220

    accuracy                           0.92      3368
   macro avg       0.68      0.64      0.65      3368
weighted avg       0.91      0.92      0.91      3368

[[2907   41   43]
 [  75   53   29]
 [  55   32  133]]
